# PDE-Net求解对流扩散方程

## 概述

PDE-Net是Zichao Long等人提出的一种前馈深度网络用于从数据中学习偏微分方程，同时实现了准确预测复杂系统的动力学特性和揭示潜在的PDE模型。PDE-Net的基本思想是通过学习卷积核(滤波器)来逼近微分算子，并应用神经网络或其他机器学习方法来拟合未知的非线性响应。数值实验表明，即使在噪声环境中，该模型也可以识别被观测的动力学方程，并预测相对较长时间的动态行为。更多信息可参考[PDE-Net: Learning PDEs from Data](https://arxiv.org/abs/1710.09668)。

本案例要求**MindSpore版本 >= 2.0.0**以调用如下接口: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*。

## 问题描述

本案例求解可变参数的对流-扩散偏微分方程的反问题，并实现长期预测。

## 控制方程

在本研究中，对流扩散方程的形式为：

$$
u_t = a(x,y) u_x + b(x,y) u_y + c u_{xx} + d u_{yy}, \quad (x,y) \in[0,2 \pi] \times[0,2 \pi]
$$

$$
u|_{t=0} = u_0(x,y)
$$

各项导数的系数分别为：

$$
a(x,y)=0.5(cos(y)+x(2\pi-x)sin(x))+0.6 \quad
b(x,y)=2(cos(y)+sin(x))+0.8
$$

$$
c=0.2 \quad
d=0.3
$$


## PDE-Net的模型结构

PDE-Net由多个$\delta T$ Block串联构成，以实现长序列信息的预测，在每一个$\delta T$ Block中，包含可训练参数的moment矩阵，该矩阵可根据映射关系转化为对应导数的卷积核，从而获取物理场的导数。将导数及其对应物理量经线性组合后，采用前向欧拉法，即可推导下一个时间步的信息。

![](images/pdenet-1.jpg)

![](images/pdenet-2.jpg)

## 技术路径

MindFlow求解该问题的具体流程如下：

1. 构建模型。
2. 单步训练。
3. 多步训练。
4. 模型推理及可视化。

In [1]:
import os
import time
import numpy as np

import mindspore
from mindspore.common import set_seed
from mindspore import nn, Tensor, context, ops, jit
from mindspore.train.serialization import load_param_into_net

In [2]:
from mindflow.cell import PDENet
from mindflow.utils import load_yaml_config
from mindflow.loss import get_loss_metric, RelativeRMSELoss
from mindflow.pde import UnsteadyFlowWithLoss

from src import init_model, create_dataset, calculate_lp_loss_error
from src import make_dir, scheduler, get_param_dic
from src import plot_coe, plot_extrapolation_error, get_label_coe, plot_test_error

In [3]:
set_seed(0)
np.random.seed(0)
context.set_context(mode=context.GRAPH_MODE, device_target="GPU", device_id=0)

In [4]:
# load configuration yaml
config = load_yaml_config('pde_net.yaml')
config["kernel_size"]=5
config["summary_dir"] = "./summary_dir/summary_kernel_5_frozen"
config["mindrecord_data_dir"] = "./data5"
config["if_frozen"] = True

## 构建模型

MindFlow提供了`PDENet`接口可以直接建立PDENet模型，需指定网格的宽度、高度、数据深度、边界条件、拟合的最高阶数等信息。

In [5]:
def init_model(config):
    return PDENet(height=config["mesh_size"],
                  width=config["mesh_size"],
                  channels=config["channels"],
                  kernel_size=config["kernel_size"],
                  max_order=config["max_order"],
                  dx=2 * np.pi / config["mesh_size"],
                  dy=2 * np.pi / config["mesh_size"],
                  dt=config["dt"],
                  periodic=config["perodic_padding"],
                  enable_moment=config["enable_moment"],
                  if_fronzen=config["if_frozen"],
                  )

## 单步训练

由于每个$\delta T$ Block的参数是共享的，因此模型根据$\delta T$ Block的串联个数依次增加，逐一进行训练。其中，在step为1时，模型处于warm-up阶段，PDE-Net的moment为“frozen”状态，此时moment中的参数不参与训练。每新增一个$\delta T$ Block，程序先进行数据生成和数据集的读取，初始化模型后，需载入前一个step训练的checkpoint，并定义优化器、模式、loss函数，并进行模型训练，在训练中模型会实时反映模型性能。

In [6]:
def train_single_step(step, config, lr, train_dataset, eval_dataset):
    """train PDE-Net with advancing steps"""

    print("Current step for train loop: {}".format(step, ))
    model = init_model(config)

    epoch = config["epochs"]
    warm_up_epoch_scale = 10
    if step == 1:
        model.if_fronzen = True
        epoch = warm_up_epoch_scale * epoch
    elif step == 2:
        param_dict = get_param_dic(config["summary_dir"], step - 1, epoch * 10)
        load_param_into_net(model, param_dict)
        print("Load pre-trained model successfully")
    else:
        param_dict = get_param_dic(config["summary_dir"], step - 1, epoch)
        load_param_into_net(model, param_dict)
        print("Load pre-trained model successfully")

    optimizer = nn.Adam(model.trainable_params(), learning_rate=Tensor(lr))
    problem = UnsteadyFlowWithLoss(model, t_out=step, loss_fn=RelativeRMSELoss(), data_format="NTCHW")

    def forward_fn(u0, uT):
        loss = problem.get_loss(u0, uT)
        return loss

    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=False)

    @jit
    def train_step(u0, uT):
        loss, grads = grad_fn(u0, uT)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    steps = train_dataset.get_dataset_size()
    sink_process = mindspore.data_sink(train_step, train_dataset, sink_size=1)

    for cur_epoch in range(epoch):
        local_time_beg = time.time()
        model.set_train()

        for _ in range(steps):
            cur_loss = sink_process()
            print("epoch: %s, loss is %s" % (cur_epoch + 1, cur_loss), flush=True)
        local_time_end = time.time()
        epoch_seconds = (local_time_end - local_time_beg) * 1000
        step_seconds = epoch_seconds / steps
        print("Train epoch time: {:5.3f} ms, per step time: {:5.3f} ms".format
              (epoch_seconds, step_seconds), flush=True)

        if (cur_epoch + 1) % config["save_epoch_interval"] == 0:
            ckpt_file_name = "ckpt/step_{}".format(step)
            ckpt_dir = os.path.join(config["summary_dir"], ckpt_file_name)
            if not os.path.exists(ckpt_dir):
                make_dir(ckpt_dir)
            ckpt_name = "pdenet-{}.ckpt".format(cur_epoch + 1, )
            mindspore.save_checkpoint(model, os.path.join(ckpt_dir, ckpt_name))

        if (cur_epoch + 1) % config['eval_interval'] == 0:
            calculate_lp_loss_error(problem, eval_dataset, config["batch_size"])

## 多步训练

PDE-Net是逐步进行训练。
使用**MindSpore>= 2.0.0**的版本，可以使用函数式编程范式训练神经网络。

In [7]:
def train(config):
    lr = config["lr"]
    for i in range(1, config["multi_step"] + 1):
        db_name = "train_step{}.mindrecord".format(i)
        dataset = create_dataset(config, i, db_name, "train", data_size=2 * config["batch_size"])
        train_dataset, eval_dataset = dataset.create_train_dataset()
        lr = scheduler(int(config["multi_step"] / config["learning_rate_reduce_times"]), step=i, lr=lr)
        train_single_step(step=i, config=config, lr=lr, train_dataset=train_dataset, eval_dataset=eval_dataset)


In [ ]:
if not os.path.exists(config["mindrecord_data_dir"]):
    make_dir(config["mindrecord_data_dir"])
train(config)

[WARNING] MD(6093,7f1ad7bcf6c0,python):2025-06-15-22:04:54.384.617 [mindspore/ccsrc/minddata/mindrecord/io/shard_writer.cc:95] OpenDataFiles] Succeed to remove the old mindrecord files, path: /home/ma-user/work/MindFlow/applications/data_mechanism_fusion/variant_linear_coe_pde_net/data5/train_step1.mindrecord
[WARNING] MD(6093,7f1ad7bcf6c0,python):2025-06-15-22:04:54.384.688 [mindspore/ccsrc/minddata/mindrecord/io/shard_writer.cc:104] OpenDataFiles] Succeed to remove the old mindrecord metadata files, path: /home/ma-user/work/MindFlow/applications/data_mechanism_fusion/variant_linear_coe_pde_net/data5/train_step1.mindrecord.db
[WARNING] ME(6093:139753265362624,MainProcess):2025-06-15-22:04:55.188.217 [mindspore/dataset/engine/datasets.py:1145] Dataset is shuffled before split.


Mindrecorder saved
Current step for train loop: 1
epoch: 1, loss is 316.3384
Train epoch time: 8298.995 ms, per step time: 8298.995 ms
epoch: 2, loss is 285.58875
Train epoch time: 10.656 ms, per step time: 10.656 ms
epoch: 3, loss is 294.71906
Train epoch time: 11.158 ms, per step time: 11.158 ms
epoch: 4, loss is 302.8824
Train epoch time: 13.838 ms, per step time: 13.838 ms
epoch: 5, loss is 298.4911
Train epoch time: 9.979 ms, per step time: 9.979 ms
epoch: 6, loss is 291.5742
Train epoch time: 12.480 ms, per step time: 12.480 ms
epoch: 7, loss is 299.95258
Train epoch time: 14.685 ms, per step time: 14.685 ms
epoch: 8, loss is 272.8629
Train epoch time: 13.964 ms, per step time: 13.964 ms
epoch: 9, loss is 300.99677
Train epoch time: 13.695 ms, per step time: 13.695 ms
epoch: 10, loss is 272.9209
Train epoch time: 7.470 ms, per step time: 7.470 ms
================================Start Evaluation================================
LpLoss_error: 16.021706
==============================